In [3]:
from fastmri.data.mri_data import fetch_dir
from fastmri.data.subsample import create_mask_for_mask_type
from fastmri.data.transforms import UnetDataTransform
from fastmri.pl_modules import FastMriDataModule, UnetModule
from fastmri.data import transforms, mri_data
import pathlib
import torch
import matplotlib.pyplot as plt
import numpy as np


CHALLENGE = 'singlecoil'
MASK_TYPE = 'random'
center_fractions = [0.08]
accelerations = [4]


mask = create_mask_for_mask_type(
        MASK_TYPE, center_fractions, accelerations
    )

train_transform = UnetDataTransform(CHALLENGE, mask_func=mask, use_seed=False)

dataset = mri_data.SliceDataset(
    root=pathlib.Path(
      './fastmri_data/singlecoil_val'
    ),
    transform=train_transform,
    challenge='singlecoil'
)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=10)
image, target, _, _, _, _, _ = next(iter(dataloader))

In [4]:

import InvertCnnConverter
import torch
import Unet

In [5]:
import importlib
importlib.reload(InvertCnnConverter)

<module 'InvertCnnConverter' from '/home/hsyang/workspace/20210411_SYS/memcnn-unet/InvertCnnConverter.py'>

In [6]:
plain_model = Unet.UNet(n_channels=1, n_classes=1)

In [7]:
plain_model

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [8]:
#InvertCnnConverter.top_forward_to_checkpoint(plain_model, last_module_name='outc')
InvertCnnConverter.convert_module(plain_model, last_module_name='outc', inplace=True)
invert_model = plain_model

device = 'cuda:0'

invert_model = invert_model.to(device)
data = image.view(-1,1,320,320).to(device)
target = target.to(device)

print(invert_model)

UNet(
  (inc): CheckpointModule(
    (module): DoubleConv(
      (double_conv): Sequential(
        (0): UpInvertibleBlock(
          (upscale): Upsample(scale_factor=(8.0, 8.0), mode=bilinear)
          (conv2): InvertibleModuleWrapper(
            (_fn): AdditiveCoupling(
              (Gm): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (Fm): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
          )
        )
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): InvertibleModuleWrapper(
          (_fn): AdditiveCoupling(
            (Gm): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (Fm): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
  )
  (down1):

In [9]:


criterion = torch.nn.MSELoss()
optim = torch.optim.Adam(invert_model.parameters(), lr=1e-3)
with torch.autograd.set_detect_anomaly(True) : 
    for epoch in range(1000) : 
        result = invert_model(data)
        loss = criterion(result, target)
        optim.zero_grad()
        loss.backward()
        optim.step()
        print(epoch, loss.item())

RestoreClass move forward/backward tensor from cpu to cuda:0
	of layer(StitchableConv2d(in_channel=1024, out_channel=512, kernel=3, stride=1, padding=1, fetch_shape=[128, 128])
# [start] StitchableConv2dFunction.forward: stitching
  alloc=612 / 1315 MB, reserved=1342 / 1342 MB 
# [end  ] StitchableConv2dFunction.forward: stitching
  alloc=612 / 1315 MB, reserved=1342 / 1342 MB 
0 10.393229484558105
# [start] StitchableConv2dFunction.forward: stitching
  alloc=616 / 3331 MB, reserved=4220 / 4220 MB 
# [end  ] StitchableConv2dFunction.forward: stitching
  alloc=616 / 3331 MB, reserved=4220 / 4220 MB 
1 10.27920913696289
# [start] StitchableConv2dFunction.forward: stitching
  alloc=616 / 3335 MB, reserved=4220 / 4220 MB 
# [end  ] StitchableConv2dFunction.forward: stitching
  alloc=616 / 3335 MB, reserved=4220 / 4220 MB 
2 10.16938304901123
# [start] StitchableConv2dFunction.forward: stitching
  alloc=616 / 3335 MB, reserved=4220 / 4220 MB 
# [end  ] StitchableConv2dFunction.forward: stit

KeyboardInterrupt: 